In [ ]:
#basic RESTful API and flask app integration with yahoo finance to pull stock data

In [ ]:
!pip install requests
!pip install flask


In [ ]:
import requests
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/stocks/<symbol>', methods=['GET'])
def get_stock_prices(symbol):
    start_date = request.args.get('start_date')
    end_date = request.args.get('end_date')
    interval = request.args.get('interval', '1d')
    url = f'https://query1.finance.yahoo.com/v7/finance/chart/{symbol}?period1={start_date}&period2={end_date}&interval={interval}'

    response = requests.get(url)
    if response.status_code != 200:
        return jsonify({'error': 'Invalid symbol or date range'}), 404
    data = response.json()
    result = {
        'symbol': symbol,
        'prices': []
    }
    for i in range(len(data['chart']['result'][0]['timestamp'])):
        result['prices'].append({
            'date': data['chart']['result'][0]['timestamp'][i],
            'open': data['chart']['result'][0]['indicators']['quote'][0]['open'][i],
            'high': data['chart']['result'][0]['indicators']['quote'][0]['high'][i],
            'low': data['chart']['result'][0]['indicators']['quote'][0]['low'][i],
            'close': data['chart']['result'][0]['indicators']['quote'][0]['close'][i],
            'volume': data['chart']['result'][0]['indicators']['quote'][0]['volume'][i]
        })
    return jsonify(result)

if __name__ == '__main__':
    app.run()


In [ ]:
#expected out example

'''

{
    "symbol": "AAPL",
    "prices": [
        {
            "date": 1514869200,
            "open": 43.064999,
            "high": 44.0625,
            "low": 43.057499,
            "close": 43.064999,
            "volume": 136811200
        },
        {
            "date": 1515474000,
            "open": 43.637501,
            "high": 44.075001,
            "low": 43.587502,
            "close": 44.052502,
            "volume": 166096800
        },
        ...
    ]
}

'''

In [ ]:
#an extra simple example

In [ ]:
import requests
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/stocks/<symbol>', methods=['GET'])
def get_stock_quote(symbol):
    response = requests.get(f'https://finance.yahoo.com/quote/{symbol}')
    if response.status_code != 200:
        return jsonify({'error': 'Invalid symbol'}), 404
    data = response.text
    quote_start = data.find('"currentPrice":') + len('"currentPrice":')
    quote_end = data.find(',', quote_start)
    quote = float(data[quote_start:quote_end])
    return jsonify({'symbol': symbol, 'quote': quote})

if __name__ == '__main__':
    app.run()
